# *Templeton project*
<img src="https://www.lymedisease.org/members/wp-content/uploads/2022/01/header-brain-health-lyme.jpg" width="380" />

## Run the update/reset shortcut on the desktop before each experiment
***
***
# **Without mouse on stage**

In [ ]:
import pathlib

import IPython
import ipywidgets as widgets

import np_logging
import np_session
import np_services
import np_workflows
from np_workflows import npxc
import np_workflows.experiments.templeton as Templeton

np_logging.getLogger()

np_workflows.elapsed_time_widget()

***
## Quiet mode
on  
*default*
- exception details are hidden
- regular messages displayed (log level = INFO)

off
- full exception details with traceback
- extra messages displayed (log level = DEBUG)

In [ ]:
np_workflows.quiet_mode_widget()

***
## Launch apps via RSC
[optional]

In [ ]:
import contextlib
with contextlib.suppress(Exception):
    np_services.start_rsc_apps()

***
## Select mouse and user

In [ ]:
user, mouse = np_workflows.user_and_mouse_widget()

***
## Select workflow
Re-run this cell if mouse ID is changed

In [ ]:
selection = Templeton.workflow_select_widget(mouse)

***
## Generate new session
Re-run this cell if workflow is changed above.

Check mouse ID and session are correct: this cell will lock them in!

In [ ]:
experiment: Templeton.Hab | Templeton.Ephys = Templeton.new_experiment(mouse, user, selection.workflow)

session: np_session.Session = experiment.session

hab: bool = isinstance(experiment, Templeton.Hab)

***
## Check task
Assign here if necessary, e.g. 

`experiment.task_name = 'templeton stage 2 aud'`

In [ ]:
experiment.task_name

***
## Pretest
Checks before running

In [ ]:
np_workflows.check_hardware_widget()

In [ ]:
if not hab:
    np_workflows.check_openephys_widget()

***
## Setup, test, reset all components
*This cell must not be skipped!*

In [ ]:
experiment.initialize_and_test_services()

***
## Run sound test

In [ ]:
experiment.run_sound_test()

***
## Dip probes

In [ ]:
if not hab:
    np_workflows.dye_widget(session.npexp_path)
    Templeton.photodoc_widget(session, 'pre_experiment_surface_image')

***
***
# **With mouse on stage**
## Before lowering cartridge

In [ ]:
experiment.log('Mouse on stage')
np_workflows.check_mouse_widget()

***
## When cartridge is lowered

In [ ]:
Templeton.photodoc_widget(session, 'brain_surface_image')

***
## Probe insertion

Temporary code for trying widget:

In [ ]:
import pathlib, IPython.display, np_probe_targets

COMBO_1 = np_probe_targets.ProbeInsertionsTempleton({
    'A':'A1',
    'B':'B1', # or B3
    'C':'C2',
    'D': None,
    'E': None,
    'F':'F1',
    }
)
COMBO_2 = np_probe_targets.ProbeInsertionsTempleton({
    'A':'A2', # or A3
    'B':'B2',
    'C':'C1', # or C4
    'D': 'D1',
    'E': None,
    'F':'F2',
    }
)
    
targets = COMBO_2

targets.save_dir = pathlib.Path('//allen/programs/mindscope/workgroups/dynamicrouting/ben')
with contextlib.suppress(Exception):
    targets.save_dir = experiment.npexp_path
    
    
IPython.display.display(
    np_probe_targets.ProbeTargetInsertionRecordWidget(
        targets, 
        implant_drawing=np_probe_targets.TempletonDrawingSVGProbeColormap, 
        # implant_drawing=np_probe_targets.TempletonDrawingSVGComboColormap, 
        current_insertion_group=np_probe_targets.ProbeInsertionsTempleton)
    )

In [ ]:
if not hab:
    np_workflows.isi_widget(mouse.lims)

***
## Photodoc before advancing probes

In [ ]:
if not hab:
    Templeton.photodoc_widget(session, 'pre_insertion_surface_image')

***
## Settle timer with dark screen (screen should be closed)

In [ ]:
if not hab:
    experiment.set_dark_desktop_on_stim()
    experiment.log('settle timer started')
    np_workflows.print_countdown_timer(minutes=.1 if experiment.workflow.value == 'pretest' else 20)
    experiment.log('settle timer finished')

***
## Use OptoGUI on Vidmon to adjust laser and save voltages
This next cell will confirm that the voltages file can be found - *optotagging
won't be able to run without it!*

In [ ]:
if not hab:
    assert pathlib.Path(experiment.optotagging_params['optoTaggingLocs']).exists(), 'voltages file not found - do not continue if optotagging is part of this experiment!'

***
## Photodoc after probes settled, before experiment

In [ ]:
if not hab:
    Templeton.photodoc_widget(session, 'post_insertion_surface_image')

In [ ]:
np_workflows.pre_stim_check_widget()

***
## Start devices recording

In [ ]:
experiment.start_recording()

Set stim desktop to grey background

In [ ]:
experiment.set_grey_desktop_on_stim()

***
## Without lick spout
### mapping | optotagging | spontaneous

In [ ]:
np_services.MouseDirector.get_proxy().retract_lick_spout()
experiment.run_mapping()
if not hab:
    experiment.run_optotagging()
    experiment.run_spontaneous()

*** 
## With lick spout
- set mouse offset
- flush line, check water delivery

In [ ]:
np_services.MouseDirector.get_proxy().extend_lick_spout()

### spontaneous + rewards | task | spontaneous + rewards
Check lickspout is extended before starting!

In [ ]:
experiment.run_spontaneous_rewards()
experiment.run_task()
if not hab:
    experiment.run_spontaneous_rewards()

***
## Without lickspout
### optotagging

In [ ]:
if not hab:
    np_services.MouseDirector.get_proxy().retract_lick_spout()
    experiment.run_optotagging()

***
## Stop recording

In [ ]:
experiment.stop_recording_after_stim_finished()
experiment.reset_desktop_on_stim()

***
## Before removing probes

In [ ]:
if not hab:
    Templeton.photodoc_widget(session, 'post_stimulus_surface_image')

***
## After fully retracting probes

In [ ]:
if not hab:
    Templeton.photodoc_widget(session, 'post_experiment_surface_image')

***
## After raising cartridge

In [ ]:
np_workflows.finishing_checks_widget()

## Record mouse off stage time

In [ ]:
experiment.log('Mouse off stage')

***

## Finalize

In [ ]:
experiment.finalize_services(*experiment.recorders, *experiment.stims)
experiment.validate_services(*experiment.recorders, *experiment.stims)

## Copy data

In [ ]:
experiment.copy_files()